## **오늘의 목표**

- Hugging Face transformers 라이브러리로 텍스트 생성 사용법 이해

- NLTK 사전 기반 단어 검증 이해

- Gradio를 활용한 간단한 웹 앱 인터페이스 구축

- 상태 기반 로직을 이해하고, 중복 방지 및 점수 시스템 구현



1. 필요한 라이브러리들을 설치해주세요.  

```!pip install transformers gradio nltk torch```

In [2]:
!pip install transformers nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [nltk]1/2 [nltk]


2. ```nltk.download("words")``` 실행하여 english_vocab 변수를 확인하세요.

In [3]:
import nltk
from nltk.corpus import words

nltk.download("words")

english_vocab = set(words.words())

print(f"단어 수: {len(english_vocab)}")
print(list(english_vocab)[:10])

단어 수: 235892
['genista', 'khediviah', 'unenraptured', 'tosticate', 'coolingly', 'Archelenis', 'divergent', 'heterochromatin', 'cartobibliography', 'unaccrued']


[nltk_data] Downloading package words to /Users/hannalee/nltk_data...
[nltk_data]   Package words is already up-to-date!


3. 사용자로부터 단어를 입력받아 입력한 단어가 포함되어 있는지 확인하는 코드를 작성하세요.

In [4]:
word = "apple"

print(word in english_vocab)


True


4. GPT로 간단하게 단어 생성 실습을 해보겠습니다.  
간단한 프롬프트를 작성하여 문장을 생성해보세요.

In [5]:
from transformers import pipeline  
generator = pipeline("text-generation", model="distilgpt2")  
prompt = "Give me a single English word that starts with 's'."
output = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.8)[0]["generated_text"]
print("\n", output)


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/hannalee/Documents/코딩/huggingface-env/lib/python3.11/site-packages/transformers/pytorch_utils.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)



 Give me a single English word that starts with 's'.
You can also see where it comes from (by the way).
I think it’s a nice move.
Here’s how I explain it:
I don’t know if there is a way to make English


5. 위 결과를 토대로 한계점을 설명하겠습니다.  

</br>

- **GPT-2 계열 (dustilgpt2)의 구조적 한계**

    - ```distilgpt2```는 **GPT-2**의 경량화 모델로, 적은 파라미터 수로 인해 정확도와 문맥 추론 능력이 제한적입니다.

    - 단어 하나만 정확히 생성하는 것보다는 문장이나 이야기 생성에 더 적합합니다.

- **프롬프트 해석의 유연함**
    - GPT 계열 모델은 **자유 생성(free generation)**을 기반으로 작동하므로,  
    ```"Give me a single English word..."``` 같은 요청을 보고도 리스트, 설명, 정의 등 다양한 방식으로 응답할 수 있습니다.

    - 인간이라면 ```"s로 시작하는 단어: sun"```처럼 줄 텐데, 모델은 정답이 아니라 문장을 생성하는 데 집중합니다.

- **정확한 제어가 어려움**
    - ```text-generation``` 파이프라인은 **지시(command)** 수행보다는 자유 텍스트 생성에 적합합니다.

    - "단어만 뽑기" 같은 작업은 ```text2text```나 ```fill-mask```, 또는 직접 후처리를 더한 커스텀 로직이 필요합니다.

6. 모델을 여러 번 로드하기 때문에 로컬에 다운로드하여 사용하겠습니다.  
지시 작업에 더 어울리는 flan-t5 계열의 모델을 다운로드하세요.

In [7]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-large"
local_path = "./models/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.save_pretrained(local_path)
model.save_pretrained(local_path)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

6-1. 다운로드가 완료된 모델을 ```mps```에 올린 뒤에 테스트를 진행해보세요.

In [12]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

device = 0 if torch.backends.mps.is_available() else -1

tokenizer = AutoTokenizer.from_pretrained(local_path)
model = AutoModelForSeq2SeqLM.from_pretrained(local_path).to("mps" if device == 0 else "cpu")

generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device)  
prompt = "Give me a single English word that starts with 's'."
output = generator(prompt, max_new_tokens=50, do_sample=True, temperature=0.8)[0]["generated_text"]
print("Result:", output)


Device set to use mps:0
/Users/hannalee/Documents/코딩/huggingface-env/lib/python3.11/site-packages/transformers/pytorch_utils.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Result: symphony


7. 원하는대로 결과가 나왔으니 끝말잇기 게임을 만들어보겠습니다. **UI**는 ```Gradio```를 활용하겠습니다.  
먼저, 단어들을 다운로드하여 소문자로 전부 변경해주세요.

In [17]:
import gradio as gr
from transformers import pipeline
import re
import nltk
from nltk.corpus import words

english_vocab = set(w.lower() for w in words.words())
english_vocab

{'genista',
 'khediviah',
 'unenraptured',
 'tosticate',
 'coolingly',
 'divergent',
 'heterochromatin',
 'cartobibliography',
 'unaccrued',
 'cion',
 'glochid',
 'anareta',
 'quintocubitalism',
 'trichology',
 'unempirical',
 'iriartea',
 'trunkmaker',
 'jakes',
 'trophically',
 'untemperamental',
 'mesian',
 'infern',
 'harpullia',
 'bonniness',
 'recoverer',
 'erythropoietic',
 'evens',
 'machineless',
 'unmicrobic',
 'overquantity',
 'castrum',
 'unobjective',
 'xanthomelanous',
 'pullable',
 'sightfulness',
 'incanton',
 'prayerfully',
 'bandless',
 'overtame',
 'nasofrontal',
 'virgin',
 'unimaginable',
 'azobenzoic',
 'hanging',
 'obliquus',
 'charleston',
 'degeneralize',
 'warsle',
 'mastoid',
 'vagodepressor',
 'haydenite',
 'fundiform',
 'groomy',
 'gaze',
 'octary',
 'inviolacy',
 'imbarn',
 'shravey',
 'unrumored',
 'yen',
 'electrologic',
 'proxy',
 'suprafoliar',
 'sawman',
 'boatloading',
 'inactively',
 'endoceratitic',
 'swimminess',
 'supersaliency',
 'macaco',
 'far

8. 사용자가 입력한 단어의 끝 알파벳만 추출하는 함수를 작성하세요.

In [25]:
def get_last_char(word):
    return word.strip()[-1].lower()

9. 입력받은 문자를 기반으로 단어 후보들을 생성한 뒤, nltk 사전으로 검증하는 함수를 작성하세요.

In [26]:
def get_candidate_words(start_char, used_words, max_trials=10):
    prompt = f"Give me an English word that starts with '{start_char}':"
    
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids=input_ids.input_ids,
        max_new_tokens=10,
        num_return_sequences=max_trials,
        do_sample=True,
        temperature=0.9
    )
    candidates = set()

    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        words_in_text = re.findall(r"\b[a-zA-Z]{3,}\b", text)
        for word in words_in_text:
            word_lower = word.lower()
            if word_lower.startswith(start_char) and word_lower in english_vocab and word_lower not in used_words:
                candidates.add(word_lower)
    return list(candidates)

    
print(get_candidate_words("banana", "", 10))

/Users/hannalee/Documents/코딩/huggingface-env/lib/python3.11/site-packages/transformers/pytorch_utils.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


['banana']


10. 함수가 정상적으로 작동하는 것을 확인했으면, 끝말잇기 게임을 실행할 수 있는 함수를 작성해보세요.

In [47]:
def play_game(user_word, state):
    if state is None:
        state = {
            "used_words": set(),
            "score_user": 0,
            "score_bot": 0,
            "last_word": None,
            "game_over": False
        }
    
    if state["game_over"]:
        return "Game Over", state
    
    if not user_word or not re.match(r"^[a-zA-Z]{3,}$", user_word):
        return "Please enter a valid English word (3+ letters).", state
    
    user_word = user_word.strip().lower()
    if user_word in state["used_words"]:
        return f"You already used '{user_word}'. Try another word.", state
    
    if user_word not in english_vocab:
        return f"{user_word} is not in the dictionaray. Try another word.", state
    
    if state["last_word"]:
        expected_start = get_last_char(state["last_word"])
        if user_word[0] != expected_start:
            return f"Your word must start with '{expected_start}'.", state
        
    state["score_user"] += 1
    state["used_words"].add(user_word)
    last_char = get_last_char(user_word)
    candidates = get_candidate_words(last_char, state["used_words"])

    if not candidates:
        state["game_over"] = True
        return f"You: {user_word}\nBot: ...\nNo more vlid words found. You Win!\nFinal Score:\nYou: {state['score_user']}\nbot: {state['score_bot']}", state
    
    bot_word = candidates[0]
    state["used_words"].add(bot_word)
    state["score_bot"] += 1
    state["last_word"] = bot_word
    return (
        f"You: {user_word}\nBot: {bot_word}\n\n"
        f"Score:\nYou: {state['score_user']}  | Bot: {state['score_bot']}",
        state
    )
    

11. 마지막으로 UI가 되어줄 Gradio 함수를 작성한 뒤, 게임이 정상적으로 실행되는지 테스트해보겠습니다.

In [48]:
# Gradio 인터페이스

iface = gr.Interface(
    fn=play_game,
    inputs=[
        gr.Textbox(label="Enter an English word"),
        gr.State()
    ],
    outputs=[
        gr.Textbox(label="Result"),
        gr.State()
    ],
    title = "English Word ChainvGame",
    description="Each word must start with the last letter of the previous word. No repeats"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


/Users/hannalee/Documents/코딩/huggingface-env/lib/python3.11/site-packages/transformers/pytorch_utils.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


Created dataset file at: .gradio/flagged/dataset1.csv
Error while flagging: Object of type set is not JSON serializable


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ CUDA 사용 (GPU):", torch.cuda.get_device_name(0))
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ MPS 사용 (Apple Silicon)")
else:
    device = torch.device("cpu")
    print("⚠️ GPU 미지원. CPU 사용 중.")

In [ ]:
import torch

# 캐시된 메모리 비우기 (PyTorch 내부 캐시)
torch.cuda.empty_cache()

# 그래픽 카드가 비어 있는지 확인
print(torch.cuda.memory_summary(device=None, abbreviated=False))